# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-mp0_h4ra
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-mp0_h4ra
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-mp0_h4ra did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-mp0_h4ra did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,d788df249976d016763ead494f53bf262c7ea4aa,389f01c14b097ed951468ff163ccc71ebcb99a27e523e9...,2014-05-09 22:45:00+00:00,2014-05-09 22:45:00+00:00,60,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
1,18b092309dff1fa8298b21f8dfb9be19124363c4,193f49f2e6f3c31a0058bb26e322f818043e6d64ff80a9...,2014-06-07 23:15:00+00:00,2014-06-07 23:15:00+00:00,0,0.0,1.703182e+10,1.703182e+10,NaN,NaN,...,NaN,NaN,Cash,None,NaN,NaN,None,NaN,NaN,None
2,7b0601c2eaeccb6c7629f48858a007afea2e98e4,2cb7deb7674470467d31b1bba4657ab1c44c1feebf6274...,2014-04-16 15:30:00+00:00,2014-04-16 15:30:00+00:00,0,0.0,NaN,NaN,NaN,77.0,...,NaN,NaN,Credit Card,None,NaN,NaN,None,41.986712,-87.663416,POINT (-87.6634164054 41.9867117999)
3,0e1861cfb146b9e1b8b773b5be89f69cee92a0fc,e055c27835840cb1b08c8f68e20066307ab235a02fe7bd...,2014-04-25 14:15:00+00:00,2014-04-25 14:15:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,Credit Card,None,NaN,NaN,None,NaN,NaN,None
4,f8583c9d5d9fde21f8bcdd7b3ed0439c21a7fd88,e055c27835840cb1b08c8f68e20066307ab235a02fe7bd...,2014-04-25 14:15:00+00:00,2014-04-25 14:15:00+00:00,60,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,Credit Card,None,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [4]:
# Fill in the blank below
avg_num_trips_query = """
        WITH trips_by_day AS
        (
            SELECT DATE(trip_start_timestamp) AS trip_date,
                COUNT(*) as num_trips
            FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
            WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
            GROUP BY trip_date
        )
        SELECT trip_date, AVG(num_trips) 
        OVER 
        (
            ORDER BY trip_date
            ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
        ) AS avg_num_trips
        FROM trips_by_day
"""

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-01-27,81378.428571
1,2016-02-14,92247.142857
2,2016-02-20,89393.428571
3,2016-01-26,82071.285714
4,2016-02-22,90943.428571


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [5]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [7]:
# Amend the query below
trip_number_query = """
        SELECT pickup_community_area, trip_start_timestamp, trip_end_timestamp,
        RANK()
        OVER (
            PARTITION BY pickup_community_area
            ORDER BY trip_start_timestamp
        ) AS trip_number
        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
        WHERE DATE(trip_start_timestamp) = '2013-10-03' 
"""

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,14.0,2013-10-03 00:00:00+00:00,2013-10-03 00:15:00+00:00,1
1,14.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
2,14.0,2013-10-03 00:15:00+00:00,2013-10-03 00:30:00+00:00,2
3,14.0,2013-10-03 00:30:00+00:00,2013-10-03 00:30:00+00:00,4
4,14.0,2013-10-03 00:45:00+00:00,2013-10-03 00:45:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [10]:
# Fill in the blanks below
break_time_query = """
        SELECT taxi_id,
            trip_start_timestamp,
            trip_end_timestamp,
            TIMESTAMP_DIFF(
                trip_start_timestamp, 
                LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                MINUTE) as prev_break
        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
        WHERE DATE(trip_start_timestamp) = '2013-10-03' 
"""

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,098c5be0c347d832b5903f8fdf5901bea4c8ba99feb5bc...,2013-10-03 20:15:00+00:00,2013-10-03 20:00:00+00:00,510.0
1,0d85210d7aaf5c6b3f6dc713e1da5b4aee0c89166c4d85...,2013-10-03 15:00:00+00:00,2013-10-03 16:00:00+00:00,225.0
2,16c0058a9dec4071ca1a3eedf10a184abdf86cde941d12...,2013-10-03 07:00:00+00:00,2013-10-03 07:15:00+00:00,255.0
3,16c0058a9dec4071ca1a3eedf10a184abdf86cde941d12...,2013-10-03 22:15:00+00:00,2013-10-03 22:15:00+00:00,435.0
4,191fd0741ff752d181e7040f03ca6025d80a0f04183be4...,2013-10-03 12:30:00+00:00,2013-10-03 12:30:00+00:00,255.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()